In [1]:
import networkx as nx
import os
import numpy

In [36]:
lattice = nx.lattice.grid_2d_graph(n=10, m=10, periodic=True)
lattice = lattice

In [37]:
def to_gset(gfile: str, g: nx.Graph):
    with open(gfile, mode='w') as gset:
        gset.write('{} {}\n'.format(g.number_of_nodes(), g.number_of_edges()))
        edgelist = g.edges()
        n = g.number_of_nodes()
        for u,v in g.edges():
            src = u[0] * 10 + u[1]
            dst = v[0] * 10 + v[1]
            gset.write('{} {} 1\n'.format(src, dst)) 

In [38]:
to_gset('../lattice_2d.gset', lattice)

In [39]:
lattice.edges()


EdgeView([((0, 0), (1, 0)), ((0, 0), (0, 1)), ((0, 0), (9, 0)), ((0, 0), (0, 9)), ((0, 1), (1, 1)), ((0, 1), (0, 2)), ((0, 1), (9, 1)), ((0, 2), (1, 2)), ((0, 2), (0, 3)), ((0, 2), (9, 2)), ((0, 3), (1, 3)), ((0, 3), (0, 4)), ((0, 3), (9, 3)), ((0, 4), (1, 4)), ((0, 4), (0, 5)), ((0, 4), (9, 4)), ((0, 5), (1, 5)), ((0, 5), (0, 6)), ((0, 5), (9, 5)), ((0, 6), (1, 6)), ((0, 6), (0, 7)), ((0, 6), (9, 6)), ((0, 7), (1, 7)), ((0, 7), (0, 8)), ((0, 7), (9, 7)), ((0, 8), (1, 8)), ((0, 8), (0, 9)), ((0, 8), (9, 8)), ((0, 9), (1, 9)), ((0, 9), (9, 9)), ((1, 0), (2, 0)), ((1, 0), (1, 1)), ((1, 0), (1, 9)), ((1, 1), (2, 1)), ((1, 1), (1, 2)), ((1, 2), (2, 2)), ((1, 2), (1, 3)), ((1, 3), (2, 3)), ((1, 3), (1, 4)), ((1, 4), (2, 4)), ((1, 4), (1, 5)), ((1, 5), (2, 5)), ((1, 5), (1, 6)), ((1, 6), (2, 6)), ((1, 6), (1, 7)), ((1, 7), (2, 7)), ((1, 7), (1, 8)), ((1, 8), (2, 8)), ((1, 8), (1, 9)), ((1, 9), (2, 9)), ((2, 0), (3, 0)), ((2, 0), (2, 1)), ((2, 0), (2, 9)), ((2, 1), (3, 1)), ((2, 1), (2, 2)), 